In [ ]:
# Reload dependencies upon changes
%load_ext autoreload
%autoreload 2

## Dependencies

In [ ]:
from functools import partial
from itertools import product, count
from copy import deepcopy
import typing
import time
import sys
import pickle

import numpy as np
import gym
import tqdm
import matplotlib.pyplot as plt

from mazelab_experimenter import EnvRegister
from mazelab_experimenter import AgentShelve
from mazelab_experimenter import benchmark, GenericOuterHook, PredictionErrorHook, apply_trace
from mazelab_experimenter.utils import compute_optimal_value


# Result output format.
FILE_FORMAT = 'pdf'
OUTPUT_FORMAT = './figures/ablation_{}.' + FILE_FORMAT

print("Output figures will be formatted as:", OUTPUT_FORMAT.format("FILENAME"))

## Environment Definition

We explore simple gridworld environments presenting tabular features to the agents. We test our agents on an empty world and on a world with rooms, we keep the dimensionality small due to memory requirements incurred by either the hindsight tables or eligibility trace models.

The agents/ environments are initialized with a binary reward function, i.e., a function that always returns $0$ until a desired state is reached upon which it returns $1$.

In [ ]:
t = 100_000  # Number of steps before terminating an episode for any agent on all environments.

name_grid10 = EnvRegister.register(
    maze_type='gridworld', 
    name=f'gridworld-v0', 
    env_args=dict(
        binary_rewards=True
    ),
    generator_args=dict(height=12, width=12),
    initialization_args=dict(
        start_pos=[[1, 1]], goal_pos=[[10, 10]]
    ),
    time_limit=t,
    override=True
)

name_grid20 = EnvRegister.register(
    maze_type='gridworld', 
    name=f'gridworld-v1', 
    env_args=dict(
        binary_rewards=True
    ),
    generator_args=dict(height=22, width=22),
    initialization_args=dict(
        start_pos=[[1, 1]], goal_pos=[[20, 20]]
    ),
    time_limit=t,
    override=True
)

name_rooms4 = EnvRegister.register(
    maze_type='n_rooms_square', 
    name=f'n_rooms-v0', 
    env_args=dict(
        binary_rewards=True
    ),
    generator_args=dict(num_rooms=4, room_size=5, gap_size=1),
    initialization_args=dict(
        start_pos=[[1, 1]], goal_pos=[[7, 7]]
    ),
    time_limit=t,
    override=True
)

name_rooms9 = EnvRegister.register(
    maze_type='n_rooms_square', 
    name=f'n_rooms-v1', 
    env_args=dict(
        binary_rewards=True
    ),
    generator_args=dict(num_rooms=9, room_size=5, gap_size=1),
    initialization_args=dict(
        start_pos=[[1, 1]], goal_pos=[[13, 13]]
    ),
    time_limit=t,
    override=True
)

name_mazeS = EnvRegister.register(
    maze_type='fixed_maze', 
    name=f'maze-v0', 
    env_args=dict(
        binary_rewards=True
    ),
    generator_args=dict(large=False),
    initialization_args=dict(
        start_pos=[[1, 1]], goal_pos=[[11, 11]]
    ),
    time_limit=t,
    override=True
)

name_mazeL = EnvRegister.register(
    maze_type='fixed_maze', 
    name=f'maze-v1', 
    env_args=dict(
        binary_rewards=True
    ),
    generator_args=dict(large=True),
    initialization_args=dict(
        start_pos=[[1, 1]], goal_pos=[[19, 19]]
    ),
    time_limit=t,
    override=True
)

# Collect each individual environment.
environment_names = [name_grid10, name_grid20, name_rooms4, name_rooms9, name_mazeS, name_mazeL]
environment_labels = ['10x10 Gridworld', '20x20 Gridworld', '4-rooms 5-to-1', '9-rooms 5-to-1', '10x10 Maze', '20x20 Maze']

In [ ]:
# Create and show an example visualization of the environment
dummy_images = list()
for name in environment_names:
    dummy = gym.make(name)
    dummy.reset()
    
    dummy_images.append(dummy.unwrapped.get_image())
    dummy.close()
    
fig, ax = plt.subplots(1, len(environment_names), figsize=(5 * len(environment_names), 5))
for i, img in enumerate(dummy_images):
    # Plot image with border padding
    ax[i].imshow(img)
    
    # Grid lines
    major, minor = np.arange(0, len(img), 1), np.arange(-.5, len(img), 1)
    ax[i].set_xticks(major); ax[i].set_yticks(major)            # Major plot ticks
    ax[i].set_xticklabels(major); ax[i].set_yticklabels(major)  # Major plot tick-labels
    ax[i].set_xticks(minor, minor=True); ax[i].set_yticks(minor, minor=True)  # Minor ticks for Grid-lines.
    
    ax[i].grid(which='minor', color='black', linestyle='-', linewidth=.5)
    
    # Annotations
    ax[i].set_title(environment_labels[i])

plt.suptitle("Benchmark environments.", fontsize=15)
plt.tight_layout()

plt.savefig(OUTPUT_FORMAT.format('BenchmarkEnvironment', FILE_FORMAT), format=FILE_FORMAT, bbox_inches="tight")
plt.show()

## Benchmark Functionality

For running the experiments, here we define helper functions that run various agent ablations over a variety of experimental parameters.

This includes functionality for logging the approximation error to the optimal Q-table. Every experiment makes use of the outlined interface.

In [ ]:
def construct_prediction_hook(_env: gym.Env, agent, solution: np.ndarray = None) -> typing.List[PredictionErrorHook]:
    """ Helper function to construct a Q-max prediction error logger for a HierQV2 based agent. """
    if solution is None:
        solution = compute_optimal_value(_env)
        
    aggr = lambda arr, i: [i, np.mean(np.square(arr)), np.mean(np.abs(arr))]
    
    references = list()
    for i in range(agent.n_levels):    
        h_a = agent.atomic_horizons[i]
        
        solution_i = solution // h_a
        reference_i = agent.discount ** solution_i 
        
        reference_i[solution == 0] = 0
    
        references.append(reference_i)
    
    hooks = list()
    for i in range(agent.n_levels):
        getter = lambda agent, i: agent.critics[i].table[:, (agent._get_index(_env.unwrapped.maze.get_end_pos()[0]) 
                                                             if agent.critics[i].goal_conditioned else 0), :].max(axis=-1)
        
        hooks.append(PredictionErrorHook(references[i], partial(getter, i=i), partial(aggr, i=i)))
        
    return hooks


def run_benchmark(r: int, it: int, eps: int, trials: int, benchmark_environments: typing.List[str], 
                  ablation_generator: typing.Callable, verbose: bool = True, 
                  **kwargs) -> typing.List[typing.Dict]:
    """ Helper function for running train-test experiments for numerous agents over a variety of experimental settings """
    benchmark_data = list()
    for env_name in benchmark_environments:
        print("=" * 50 + "\nExperiment environment:", env_name, end='\n\n')
        meta, agent_gens, separators = ablation_generator(env_name)

        env_data = list()
        for i, (m, gen_a) in enumerate(zip(meta, agent_gens)):
            print(f"Benchmarking agent: {i+1}/{len(meta)} - ablation:", m)

            # Run the benchmark
            env_data.append(
                benchmark(
                    env_id=env_name, 
                    _agent_gen=gen_a, 
                    agent_test_kwargs=dict(behaviour_policy=False),
                    agent_train_kwargs=dict(),
                    skip_random_evaluation=True,
                    num_repetitions=r,
                    num_iterations=it,
                    num_episodes=eps,
                    num_trials=trials,
                    evaluation_hooks=[GenericOuterHook()],
                    verbose=verbose,
                    asynchronous=False,
                    **kwargs
                )
            )

        benchmark_data.append({'name': env_name, 'meta': meta, 'data': env_data, 'separators': separators})
        
    return benchmark_data



def run_episode(r: int, benchmark_environments: typing.List[str], ablation_generator: typing.Callable, 
                verbose: bool = True, **kwargs) -> typing.List[typing.Dict]:
    """ Helper function for training numerous agents once over a variety of experimental settings """
    benchmark_data = list()
    for env_name in benchmark_environments:
        print("=" * 50 + "\nExperiment environment:", env_name, end='\n\n')
        meta, agent_gens, separators = ablation_generator(env_name)

        env_data = list()
        for i, (m, gen_a) in enumerate(zip(meta, agent_gens)):
            print(f"Train agent: {i+1}/{len(meta)} - ablation:", m)
            
            rep_data = list()
            for _ in tqdm.trange(r):
                env = gym.make(env_name)
                agent = gen_a()

                agent.reset()
                agent.train(env, 1, False)
                rep_data.append(env.steps)

            env_data.append(rep_data)
                
        benchmark_data.append({'name': env_name, 'meta': meta, 'data': env_data, 'separators': separators})
        
    return benchmark_data


## Experiment Performance Multi-Step Returns: Agent Definition 

For benchmarking the agents we utilize two types of implemented classes for either the flat or the hierarchical agents. These are the atomic and compound update types, note that an atomic 1-step update $\equiv$ to a $\lambda=0$ compound update which is just 1-step $TD$. For $\lambda=0$ we can fall back to the multi-step implementation with $n=1$ for simplicity, we test out both methods to assert empirical equivalence.

In [ ]:
# Shared Keyword arguments
shared_kwargs = {
    'epsilon': 0.25,
    'lr': 1.0
}

# Defaults for Hierarchical agents.
hierarchical_defaults = {
    'horizons': 3
}


def create_agent_generator(_env: gym.Env, atomic: bool, **kwargs) -> typing.Callable:
    """ Helper function for instantiating agents with a fixed goal given some environment. 
    
    Atomic implies n-step returns, otherwise implies compound/ lambda returns.
    Hierarchical denotes whether to use a hierarchical Q-learner or a flat agent.
    
    Actual agent hyperparameters 'v' should be specified outside this function.
    """
    agent = AgentShelve.retrieve(
        agent=('HierQN' if atomic else 'HierQLambda'),  
        keyword_arguments=dict(
            observation_shape=_env.observation_space.shape,
            n_actions=_env.action_space.n,
            **kwargs, **hierarchical_defaults, **shared_kwargs
        ))
    agent.set_goal(agent._get_index(_env.unwrapped.maze.get_end_pos()[0]))   
    return agent


def get_environment_agent_ablations(_env_name: str, ablation_grid: typing.Dict) -> typing.List[typing.Generator]:
    """ Helper function for creating agent generators with an environment dependency over a grid of parameters. """
    dummy = gym.make(_env_name)
    
    # Unpack all ablation-runs over their parameter grid into flat lists.
    meta, agent_generators, separators = list(), list(), list()
    for name, ablations in ablations_grid.items():
        # Take cartesian product of all specified parameter options.
        keys, values = zip(*ablations.items())
        grid = [dict(zip(keys, v)) for v in product(*values)]
        
        # Initialize agents.
        for p in grid:
            gen = partial(create_agent_generator, _env=dummy, **p)
       
            # Add agent generator and ablation metadata to global execution list.        
            agent_generators.append(gen)
            meta.append(p)
        
        # Append grid length to separate list for slicing convenience.
        separators.append(len(grid))
        
    return meta, agent_generators, separators
        

# Define (ad-hoc) Parameter Ablation Grid and wrap into generator function.
ablations_grid = {
    'lambda': {
        'atomic': [False], 'discount': [0.95],'decay': [0.0, 0.5, 0.8, 1.0], 
        'n_levels': [4, 3, 2, 1], 'greedy_options': [True], 'flat_training': [True, False]},
    'multistep_hierarchical': {
        'atomic': [True], 'discount': [0.95], 
        'n_steps': [1, 3, 5, 8], 'n_levels': [4, 3, 2, 1], 'greedy_options': [True], 'flat_training': [True, False]}
}
# Use this function with an environment arguments to get all agents specified by the parameter grid as generators.
# >> meta, agents, seps = get_environment_ablations(env_name)
get_environment_ablations = partial(get_environment_agent_ablations, ablation_grid=ablations_grid)

### Run Benchmark & IO

Collect number of steps needed in first training episode to observe first environment reward.

In [ ]:
r = 200

for e in environment_names:  # The small environments
    
    results = run_episode(
        r=r,
        benchmark_environments=[e],  
        ablation_generator=get_environment_ablations,
        verbose=True
    )
    
    with open(f't100k_first_reward_ablation_{e}_{int(time.time())}.out', 'wb') as f:
        pickle.dump(results, f)

**Run Small Environments**

Collect performance over multiple training episodes

In [ ]:
r = 200

for e in environment_names[::2]:  # The small environments
    # Benchmark configuration and execution.
    small_benchmark_data = run_benchmark(
        r=r,
        it=50,
        eps=1,
        trials=1,
        benchmark_environments=[e],  
        ablation_generator=get_environment_ablations,
        verbose=True
    )

    with open(f't100kperformance_ablation_{e}_{int(time.time())}.out', 'wb') as f:
        pickle.dump(small_benchmark_data, f)
    

**Run Large Environments**

In [ ]:
r = 200

for e in environment_names[1::2]:  # The Large environments
    # Benchmark configuration and execution.
    large_benchmark_data = run_benchmark(
        r=r,
        it=50,
        eps=1,
        trials=1,
        benchmark_environments=[e], 
        ablation_generator=get_environment_ablations,
        verbose=True
    )

    with open(f'performance_ablation_{e}_{int(time.time())}.out', 'wb') as f:
        pickle.dump(large_benchmark_data, f)
        
    break

## Discount Balancing HierQ Ablation

In [ ]:
# Calibration reference: 2-level hierarchy with H=3 and discount=0.95
base_discount = 0.95
base_horizon = 3.0
base_nstep = 3
base_level = 2.0

# Grid for balancing effective credit assignment depth for different hierarchy levels:
levels = np.asarray([1, 2, 3])

gammas = base_discount ** (base_horizon ** (levels - base_level))
nsteps = base_nstep // (base_horizon ** (levels - base_level))
print("Adjusted discounts to balance reward propagation depth:\n", *zip(levels, gammas.round(4)))
print("Adjusted multistep to balance reward propagation depth:\n", *zip(levels, nsteps))

In [ ]:
# Shared Keyword arguments
shared_kwargs = {
    'epsilon': 0.25,
    'lr': 1.0
}

# Defaults for Hierarchical agents.
hierarchical_defaults = {
    'horizons': 3
}

# Generate Parameter Ablation Grid based on adjusted discount-parameters per hierarchy level, and wrap into generator function.
ablations_grid = dict()

base_lambda = {'flat_training': [True, False], 'atomic': [False], 'decay': [1.0], 'greedy_options': [True]}
base_nstep = {'flat_training': [True, False], 'atomic': [True], 'greedy_options': [True]}
for i, discount, n_adjusted in zip(levels, gammas, nsteps):
    p = {'discount': [discount], 'n_levels': [i]}
    
    l, n = base_lambda.copy(), base_nstep.copy()
    
    l.update(p); n.update(p)
    n['n_steps'] = [int(n_adjusted)]
    
    if i == 1:
        l['flat_training'] = [True]
        n['flat_training'] = [True]
    
    ablations_grid[f'multistep_i={i}'] = n
    ablations_grid[f'lambda_i={i}'] = l

In [ ]:
# Check for correctness
ablations_grid

In [ ]:
# Use this function with an environment arguments to get all agents specified by the parameter grid as generators.
# >> meta, agents, seps = get_environment_ablations(env_name)
get_environment_ablations = partial(get_environment_agent_ablations, ablation_grid=ablations_grid)

r = 200

for e in environment_names:  # The Large environments
    # Benchmark configuration and execution.
    large_benchmark_data = run_benchmark(
        r=r,
        it=50,
        eps=1,
        trials=1,
        benchmark_environments=[e], 
        ablation_generator=get_environment_ablations,
        verbose=True
    )

    with open(f't100k-full-discount-performance_ablation_{e}_{int(time.time())}.out', 'wb') as f:
        pickle.dump(large_benchmark_data, f)
        